In [1]:
import os
import pandas as pd

In [2]:
BASE_PATH = '/'.join(os.getcwd().split('/')[:-1]) + '/'
RESULTS_PATH = BASE_PATH +  'results/'
EMISSIONS_FILE = 'emissions.csv'
METRICS_FILE = 'metrics.csv'
PARAMS_FILE = 'params.csv'

In [3]:
experiments = []

for d in next(os.walk(RESULTS_PATH))[1]:
	for m in next(os.walk(RESULTS_PATH + d))[1]:
		experiments.append(os.path.join(d, m))

experiments = sorted(experiments)

In [4]:
for i, v in enumerate(experiments):
	print(f'{i+1:<4}{v:<15}')

1   movielens/BPR  
2   movielens/CFKG 
3   movielens/Pop  


In [5]:
emissions = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], EMISSIONS_FILE))
metrics = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], METRICS_FILE))
params = pd.read_csv(os.path.join(RESULTS_PATH, experiments[0], PARAMS_FILE))

In [6]:
emissions.head()

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2023-11-24T10:09:22,MOVIELENS_BPR_PARAMS_TUNING_1700816558,0351208f-17ba-4299-8ea1-946b1f6fefcb,95.395872,0.000037,3.917708e-07,0.168900,0.0079,0.403879,0.000103,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
1,2023-11-24T10:13:00,MOVIELENS_BPR_PARAMS_TUNING_1700816558,14e62635-c841-4a4e-a852-d7a9d7f5cf32,203.982522,0.000092,4.498937e-07,5.527600,0.3164,0.000000,0.000253,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
2,2023-11-24T10:14:40,MOVIELENS_BPR_PARAMS_TUNING_1700816558,20da207a-73ca-437c-90cc-89f63a329e5c,83.937617,0.000039,4.693520e-07,4.721833,0.1755,0.000000,0.000108,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
3,2023-11-24T10:16:23,MOVIELENS_BPR_PARAMS_TUNING_1700816558,1078ad49-9247-4891-a77e-462be623c7f4,88.991596,0.000037,4.144403e-07,0.321000,0.0442,0.451092,0.000102,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0
4,2023-11-24T10:24:31,MOVIELENS_BPR_PARAMS_TUNING_1700816558,b2f67b5c-830a-4d5f-8098-61bab9f68cf2,472.005726,0.000196,4.148935e-07,0.300800,0.0026,0.470369,0.000532,...,12,Apple M2 Max,1,Apple M2 Max,16.3097,41.2325,64.0,process,N,1.0


In [7]:
metrics.head()

,recall@10,mrr@10,ndcg@10,hit@10,map@10,precision@10,gauc,itemcoverage@10,averagepopularity@10,giniindex@10,shannonentropy@10,tailpercentage@10,run_id,project_name
0,0.1356,0.3850,0.2098,0.6970,0.1152,0.1658,0.9154,0.5657,1052.6714,0.8550,0.0031,0.0000,0351208f-17ba-4299-8ea1-946b1f6fefcb,MOVIELENS_BPR_PARAMS_TUNING_1700816558
1,0.1600,0.4346,0.2522,0.7336,0.1493,0.2000,0.9263,0.4535,1166.6326,0.8928,0.0037,0.0000,14e62635-c841-4a4e-a852-d7a9d7f5cf32,MOVIELENS_BPR_PARAMS_TUNING_1700816558
2,0.1383,0.4028,0.2258,0.6949,0.1303,0.1808,0.9186,0.3906,1273.9358,0.9198,0.0040,0.0000,20da207a-73ca-437c-90cc-89f63a329e5c,MOVIELENS_BPR_PARAMS_TUNING_1700816558
3,0.1383,0.4028,0.2258,0.6949,0.1303,0.1808,0.9186,0.3906,1273.9358,0.9198,0.0040,0.0000,1078ad49-9247-4891-a77e-462be623c7f4,MOVIELENS_BPR_PARAMS_TUNING_1700816558
4,0.1220,0.3976,0.2142,0.6561,0.1252,0.1696,0.9097,0.2126,1669.2791,0.9712,0.0061,0.0002,b2f67b5c-830a-4d5f-8098-61bab9f68cf2,MOVIELENS_BPR_PARAMS_TUNING_1700816558


In [8]:
params.head()

,gpu_id,worker,use_gpu,seed,state,reproducibility,data_path,checkpoint_dir,show_progress,save_dataset,...,entity_kg_num_interval,relation_kg_num_interval,benchmark_filename,MODEL_TYPE,encoding,dataset,model,device,run_id,project_name
0,0,0,True,42,INFO,True,data/,saved/movielens/BPR,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.GENERAL,utf-8,movielens,BPR,mps,0351208f-17ba-4299-8ea1-946b1f6fefcb,MOVIELENS_BPR_PARAMS_TUNING_1700816558
1,0,0,True,42,INFO,True,data/,saved/movielens/BPR,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.GENERAL,utf-8,movielens,BPR,mps,14e62635-c841-4a4e-a852-d7a9d7f5cf32,MOVIELENS_BPR_PARAMS_TUNING_1700816558
2,0,0,True,42,INFO,True,data/,saved/movielens/BPR,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.GENERAL,utf-8,movielens,BPR,mps,20da207a-73ca-437c-90cc-89f63a329e5c,MOVIELENS_BPR_PARAMS_TUNING_1700816558
3,0,0,True,42,INFO,True,data/,saved/movielens/BPR,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.GENERAL,utf-8,movielens,BPR,mps,1078ad49-9247-4891-a77e-462be623c7f4,MOVIELENS_BPR_PARAMS_TUNING_1700816558
4,0,0,True,42,INFO,True,data/,saved/movielens/BPR,True,False,...,"[0,inf)","[0,inf)",NaN,ModelType.GENERAL,utf-8,movielens,BPR,mps,b2f67b5c-830a-4d5f-8098-61bab9f68cf2,MOVIELENS_BPR_PARAMS_TUNING_1700816558


In [9]:
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1])+'/src')
from config.global_config import get_global_config
from utils import get_total_iterations
_config = get_global_config()
MODELS = sorted(_config.get('MODELS'))
BASE_PATH = '/'.join(os.getcwd().split('/')[:-1])
HP_CONFIG_PATH = os.path.join(BASE_PATH, _config.get('HP_CONFIG_PATH'))

In [10]:
print('\n' + ''.join(['> ' for i in range(30)]))
print(f'\n{"MODEL":<20}{"N_ITER":<15}\n')
print(''.join(['> ' for i in range(30)]))
for m in MODELS:
	n = get_total_iterations(HP_CONFIG_PATH + m + ".hyper")
	print(f'{m:<20}{n:<15}')


> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

MODEL               N_ITER         

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
ADMMSLIM            20736          
BPR                 27             
CDAE                1296           
CFKG                162            
CKE                 243            
ConvNCF             20736          
DGCF                243            
DMF                 81             
DiffRec             2985984        
EASE                54             
ENMF                2592           
FISM                54             
GCMC                1944           
ItemKNN             216            
KGAT                18225          
KGCN                729            
KGIN                1296           
KGNNLS              162            
KTUP                23328          
LDiffRec            191102976      
LINE                162            
LightGCN            216            
MCCLK               6480           
MKR        